In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials 
# spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

#uncomment above and replace CLIENT_ID and CLIENT_SECRET with the id and secret you get from creating your app with spotify (going through the "Getting Started" instructions in the Spotify Web API documentation)

from sklearn.model_selection import train_test_split
import math

/Users/yush/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Spotify Integration

In [ ]:
discussions_df = pd.read_csv('data/Discussions.csv', header=0, encoding='utf-8')
ratings_df = pd.read_csv('data/Ratings.csv', header=0, encoding='utf-8')
display(discussions_df)
display(ratings_df)

In [ ]:
discussions_df['EnergyAvg'] = 0
discussions_df['ValenceAvg'] = 0
discussions_df['AcousticnessAvg'] = 0
discussions_df['InstrumentalnessAvg'] = 0

for ind in range(len(discussions_df)):
    results = spotify.album_tracks(discussions_df['SpotifyID'][ind])

    ids = []
    for track in results['items']:
        ids.append(track['id'])
    numTracks = len(ids)

    features = spotify.audio_features(ids)
    for feature in features['audio_features']:
        discussions_df['EnergyAvg'][ind] += feature['energy'] / numTracks
        discussions_df['ValenceAvg'][ind] += feature['valence'] / numTracks
        discussions_df['AcousticnessAvg'][ind] += feature['acousticness'] / numTracks
        discussions_df['InstrumentalnessAvg'][ind] += feature['instrumentalness'] / numTracks
    
display(discussions_df)

# Your Turn
Fill out the the database with audio features (watch out for 429 errors!) and run some of the filtering techniques we've covered on it.

Content filtering is a great fit here

also be cognisant of the fact that audio features are numerical values, and thus can be leveraged in weighted sums, numerical analysis, and any number of data science techniques.

and like mentioned in the content filtering slides, feel free to go above and beyond with any external resoruces (like RateYourMusic, AlbumOfTheYear, etc.) you'd like!

In [1]:
# TODO: integrate Spotify with your filtering code